### Import Dependencies

In [3]:
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
import os

from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents.run_config import RunConfig
from google.genai import types

from utils.tools import check_warehouse_availability, reserve_warehouse_items

### ADK Agent

In [4]:
model = LiteLlm(
    model="openai/gpt-4.1-mini",
    temperature=0.0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [5]:
warehouse_agent = Agent(
    name="warehouse_manager_agent",
    model=model,
    tools=[check_warehouse_availability, reserve_warehouse_items],
    description="A agent that can check the availability of items in the warehouses and reserve them.",
    instruction="""
You are a part of the shopping assistant that can manage available inventory in the warehouses.

You will be given a conversation history and a list of tools, your task is to perform actions requested by the latest user query. Answer part of the query that you can answer with the available tools.

Instructions:
- You must always check the availability of the items in the warehouses before reserving them.
- Only reserve items in warehouses if entire order can be reserved or the user has confirmed that they want a partial reservation.
- If you cannot reserve any items, return an answer that the order cannot be reserved.
- If you can reserve some items, return an answer that the order can be partially reserved and include the details.
- If only partial quantity can be reserved in some warehouses, try to combinethe required quantity from different warehouses.
- Try to reserve items from the closest warehouse to the user first if users location is provided.
"""
)

### ADK Session

In [6]:
session_service = InMemorySessionService()

In [7]:
await session_service.create_session(
    app_name="warehouse_app",
    user_id="user_1",
    session_id="session_1"
)

Session(id='session_1', app_name='warehouse_app', user_id='user_1', state={}, events=[], last_update_time=1772256720.9132328)

In [8]:
runner = Runner(
    agent=warehouse_agent,
    session_service=session_service,
    app_name="warehouse_app",
)

In [9]:
message = types.Content(
    role="user",
    parts=[types.Part(text="What is the availability of B09WCL37Z4 in all of the warehouses?")]
)

In [10]:
result = runner.run(
    user_id="user_1",
    session_id="session_1",
    new_message=message,
    run_config=RunConfig(
        max_llm_calls=3,
    )
)

In [11]:
for event in result:
    print("===========================================")
    print(event)

model_version='gpt-4.1-mini-2025-04-14' content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'items': [
            {<... 2 items at Max depth ...>},
          ]
        },
        id='call_cDO9Nx52c4szaha21RLtRV2t',
        name='check_warehouse_availability'
      )
    ),
  ],
  role='model'
) grounding_metadata=None partial=False turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  cached_content_token_count=0,
  candidates_token_count=32,
  prompt_token_count=510,
  total_token_count=542
) live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None interaction_id=None invocation_id='e-a939e9a6-2f9e-4a52-a946-521822ae6209' author='warehouse_manager_agent' actions=EventActions(skip_summarization=None

In [ ]:
session_service = InMemorySessionService()

async def ask_warehouse_agent(query: str, session_id: str):

    # Check if session exists, only create if it doesn't
    existing_session = await session_service.get_session(
        app_name="warehouse_app",
        user_id="user_1",
        session_id=session_id,
    )

    if not existing_session:
        await session_service.create_session(
            app_name="warehouse_app",
            user_id="user_1",
            session_id=session_id,
        )

    runner = Runner(
        agent=warehouse_agent,
        app_name="warehouse_app",
        session_service=session_service,
    )

    content = types.Content(
        role="user",
        parts=[types.Part(text=query)],
    )

    final_text = ""

    for event in runner.run(
        user_id="user_1",
        session_id=session_id,
        new_message=content,
        run_config=RunConfig(
            max_llm_calls=3,
        ),
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                for part in event.content.parts:
                    final_text += part.text
            break

    return final_text

In [13]:
answer = await ask_warehouse_agent("What is the availability of B09WCL37Z4 in all of the warehouses?", session_id = "123")

In [14]:
print(answer)

The product B09WCL37Z4 is available in the following warehouses with full availability for at least 1 unit:

- Berlin Distribution Center (Berlin, Germany): 72 units available
- Munich Logistics Hub (Munich, Germany): 23 units available
- Paris Central Depot (Paris, France): 9 units available
- Marseille Mediterranean Hub (Marseille, France): 4 units available

It is not available in Lyon Regional Warehouse (Lyon, France) and Hamburg North Warehouse (Hamburg, Germany). Let me know if you want to reserve any quantity from these warehouses.


In [15]:
answer = await ask_warehouse_agent("Can you reserve 6 in Lyon", session_id = "123")

In [16]:
answer

'The product B09WCL37Z4 is not available in the Lyon Regional Warehouse, so I cannot reserve 6 units from there. Would you like to reserve from another warehouse instead?'

In [17]:
answer = await ask_warehouse_agent("Can you reserve 6 in Berlin", session_id = "123")

In [18]:
answer

'I have successfully reserved 6 units of product B09WCL37Z4 in the Berlin Distribution Center. If you need any further assistance, please let me know!'